<a href="https://colab.research.google.com/github/snehastar/Machine-Learning/blob/main/Covid_19_exploratory_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the libraries

In [1]:

from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

#Importing the dataset

In [2]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

#Viewing the data

In [3]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,43240,43468,43681,43924,44177,44363,44503,44706,44988,45174,45384,45600,45723,45844,46116,46274,46516,46718,46837,46837,47072,47306,47516,47716,47851,48053,48116,48229,48527,48718,48952,49161,49378,49621,49681,49817,50013,50190,50433,50655
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,27830,28432,29126,29837,30623,31459,32196,32761,33556,34300,34944,35600,36245,36790,37625,38182,39014,39719,40501,41302,42148,42988,43683,44436,45188,46061,46863,47742,48530,49191,50000,50637,51424,52004,52542,53003,53425,53814,54317,54827
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,67679,68589,69591,70629,71652,72755,73774,74862,75867,77000,78025,79110,80168,81212,82221,83199,84152,85084,85927,86730,87502,88252,88825,89416,90014,90579,91121,91638,92102,92597,93065,93507,93933,94371,94781,95203,95659,96069,96549,97007
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5872,5914,5951,6018,6066,6142,6207,6256,6304,6351,6428,6534,6610,6610,6712,6745,6790,6842,6904,6955,7005,7050,7084,7127,7162,7190,7236,7288,7338,7382,7382,7446,7466,7519,7560,7577,7602,7633,7669,7699
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13451,13615,13818,13922,14134,14267,14413,14493,14634,14742,14821,14920,15008,15087,15103,15139,15251,15319,15361,15493,15536,15591,15648,15729,15804,15925,16061,16161,16188,16277,16362,16407,16484,16562,16626,16644,16686,16802,16931,17029


In [4]:
recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,35092,35137,35160,35295,35350,35370,35422,35934,35976,36122,36145,36232,36295,36709,36716,36831,36946,37218,37260,37260,37393,37685,37879,37920,38032,38099,38141,38200,38250,38252,38336,38475,38505,38540,38613,39006,39508,39585,39692,40359
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,12889,13453,13804,14216,14565,15055,15469,15842,16230,16666,17031,17352,17755,18152,18481,18849,19384,19912,20484,20974,21286,21617,22180,22527,23072,23609,24136,24520,24820,24820,25876,26381,26898,27426,27831,28121,28752,29249,29799,30276
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,44633,45148,45148,46326,46962,47581,48183,48794,49421,50070,50712,51334,51946,52568,53204,53809,54405,54990,55538,56079,56617,57146,57648,58146,58146,59135,59590,60028,60457,60888,61307,61700,62089,62487,62869,63260,63644,64020,64401,64777
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4747,4830,4965,5055,5127,5239,5290,5358,5405,5503,5542,5649,5710,5710,5794,5873,5940,5988,6066,6130,6171,6238,6293,6367,6452,6505,6598,6629,6629,6706,6706,6819,6875,6919,6963,6997,7028,7073,7106,7171
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6444,6523,6582,6623,7062,7117,7273,7346,7351,7444,7517,7617,7697,7763,7763,7851,7932,8139,8244,8299,8335,8338,8353,8470,8579,8679,8798,8841,8898,8924,8990,9194,9266,9345,9518,9592,9695,9729,9729,9729


In [5]:
death_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1620,1635,1641,1648,1653,1666,1678,1690,1702,1715,1728,1737,1740,1752,1774,1795,1822,1841,1846,1846,1864,1874,1900,1906,1919,1935,1945,1956,1965,1969,1995,2011,2025,2030,2047,2067,2082,2096,2117,2126
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,623,631,637,646,657,672,685,699,716,735,743,753,771,787,798,810,822,839,852,870,889,905,922,936,951,965,977,989,1003,1016,1028,1040,1055,1066,1074,1088,1098,1111,1117,1125
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2154,2168,2186,2206,2224,2236,2255,2272,2294,2309,2329,2352,2372,2393,2410,2431,2447,2464,2480,2492,2501,2516,2527,2539,2554,2564,2575,2584,2596,2609,2623,2631,2640,2647,2659,2666,2675,2687,2696,2705
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76,76,77,77,78,78,78,78,78,78,78,78,79,79,79,79,79,80,80,81,81,82,82,83
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,322,324,328,332,333,334,336,337,337,338,340,341,342,345,346,348,350,351,352,353,354,354,354,355,358,362,365,366,371,372,372,379,382,384,386,387,390,393,393,393


In [6]:
country_df.head()

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Afghanistan,2020-12-25 12:22:39,33.93911,67.709953,50810.0,2139.0,40444.0,8227.0,130.521874,NaN,NaN,4.209801,4,AFG
1,Albania,2020-12-25 12:22:39,41.15330,20.168300,54827.0,1125.0,30276.0,23426.0,1905.170616,NaN,NaN,2.051909,8,ALB
2,Algeria,2020-12-25 12:22:39,28.03390,1.659600,97007.0,2705.0,64777.0,29525.0,221.219368,NaN,NaN,2.788459,12,DZA
3,Andorra,2020-12-25 12:22:39,42.50630,1.521800,7699.0,83.0,7171.0,445.0,9964.408206,NaN,NaN,1.078062,20,AND
4,Angola,2020-12-25 12:22:39,-11.20270,17.873900,17029.0,393.0,9729.0,6907.0,51.813002,NaN,NaN,2.307828,24,AGO


#Data cleaning

Converting headings to lower case

In [7]:
country_df.columns = map(str.lower, country_df.columns)
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

In [8]:
country_df.isnull().sum()

country_region           0
last_update              0
lat                      2
long_                    2
confirmed                0
deaths                   0
recovered                4
active                   0
incident_rate            2
people_tested          191
people_hospitalized    191
mortality_rate           0
uid                      0
iso3                     2
dtype: int64

Renaming some of the columns

In [9]:
death_df = death_df.rename(columns = {'province/state' : 'state' , 'country/region' : 'country'})
confirmed_df = confirmed_df.rename(columns = {'province/state' : 'state' , 'country/region' : 'country'})
recovered_df = recovered_df.rename(columns = {'province/state' : 'state' , 'country/region' : 'country'})
country_df = country_df.rename(columns = {'country_region' : 'country'})

#Analyzing the data

Sorting the data on the basis of confirmed cases

In [10]:
sorted_country_df = country_df.sort_values('confirmed', ascending = False)
sorted_country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
177,US,2020-12-25 12:22:39,40.000000,-100.00000,18655560.0,329106.0,NaN,18326323.0,5662.357869,NaN,NaN,1.764118,840,USA
79,India,2020-12-25 12:22:39,20.593684,78.96288,10146845.0,147092.0,9717834.0,281919.0,735.276287,NaN,NaN,1.449633,356,IND
23,Brazil,2020-12-25 12:22:39,-14.235000,-51.92530,7423945.0,189982.0,6512717.0,721246.0,3492.644732,NaN,NaN,2.559044,76,BRA
141,Russia,2020-12-25 12:22:39,61.524000,105.31880,2963290.0,52985.0,2374589.0,535716.0,2030.562213,NaN,NaN,1.788046,643,RUS
62,France,2020-12-25 12:22:39,46.227600,2.21370,2584471.0,62389.0,194559.0,2327385.0,3959.448359,NaN,NaN,2.413995,250,FRA


Highlighting the important columns

In [11]:
def highlight_col(x):
  p = 'background-color : pink'
  y = 'background-color : yellow'
  g = 'background-color : skyblue'
  temp_df = pd.DataFrame('', index = x.index, columns= x.columns)
  temp_df.iloc[: , 4] = p
  temp_df.iloc[: , 5] = y
  temp_df.iloc[: , 6] = g
  return temp_df
sorted_country_df.head(5).style.apply(highlight_col, axis = None)

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
177,US,2020-12-25 12:22:39,40.000000,-100.000000,18655560.000000,329106.000000,nan,18326323.000000,5662.357869,nan,nan,1.764118,840,USA
79,India,2020-12-25 12:22:39,20.593684,78.962880,10146845.000000,147092.000000,9717834.000000,281919.000000,735.276287,nan,nan,1.449633,356,IND
23,Brazil,2020-12-25 12:22:39,-14.235000,-51.925300,7423945.000000,189982.000000,6512717.000000,721246.000000,3492.644732,nan,nan,2.559044,76,BRA
141,Russia,2020-12-25 12:22:39,61.524000,105.318800,2963290.000000,52985.000000,2374589.000000,535716.000000,2030.562213,nan,nan,1.788046,643,RUS
62,France,2020-12-25 12:22:39,46.227600,2.213700,2584471.000000,62389.000000,194559.000000,2327385.000000,3959.448359,nan,nan,2.413995,250,FRA


#Visualize the data

In [12]:
fig = px.scatter(sorted_country_df.head(15) , x = 'country', y = 'confirmed', size = 'confirmed', color = 'country',
      hover_name = "country", size_max = 70)
fig.show()

In [13]:
def plot_cases_for_country(country):
  labels = ['confirmed' , 'deaths']
  colors = ['blue' , 'red']
  mode_size = [6,8]
  line_size = [4,5]
  df_list = [confirmed_df, death_df]
  fig = go.Figure()

  for i, df in enumerate(df_list):
    if country == 'World' or country == 'world':
      x_data = np.array(list(df.iloc[:,5:].columns))
      y_data = np.sum(np.array(df.iloc[: , 5:]), axis =0)
    else:
      x_data = np.array(list(df.iloc[:,5:].columns))
      y_data = np.sum(np.asarray(df[df['country']==country].iloc[: , 5:]), axis = 0)

    fig.add_trace(go.Scatter(x = x_data, y = y_data, mode = 'lines+markers',
                             name = labels[i], line = dict(color = colors[i],width = line_size[i]),
                             connectgaps = True, 
                             text = "Total" + str(labels[i]) + ": "+ str(y_data[-1])
                             ))
    fig.show()
#plot_cases_for_country('India')
interact(plot_cases_for_country, country = 'World')

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

<function __main__.plot_cases_for_country>

In [14]:

df1 = confirmed_df.dropna(subset=['lat','long'], axis=0 )
df2 = death_df.dropna(subset=['lat','long'], axis=0 )
df1.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,43240,43468,43681,43924,44177,44363,44503,44706,44988,45174,45384,45600,45723,45844,46116,46274,46516,46718,46837,46837,47072,47306,47516,47716,47851,48053,48116,48229,48527,48718,48952,49161,49378,49621,49681,49817,50013,50190,50433,50655
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,27830,28432,29126,29837,30623,31459,32196,32761,33556,34300,34944,35600,36245,36790,37625,38182,39014,39719,40501,41302,42148,42988,43683,44436,45188,46061,46863,47742,48530,49191,50000,50637,51424,52004,52542,53003,53425,53814,54317,54827
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,67679,68589,69591,70629,71652,72755,73774,74862,75867,77000,78025,79110,80168,81212,82221,83199,84152,85084,85927,86730,87502,88252,88825,89416,90014,90579,91121,91638,92102,92597,93065,93507,93933,94371,94781,95203,95659,96069,96549,97007
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,5872,5914,5951,6018,6066,6142,6207,6256,6304,6351,6428,6534,6610,6610,6712,6745,6790,6842,6904,6955,7005,7050,7084,7127,7162,7190,7236,7288,7338,7382,7382,7446,7466,7519,7560,7577,7602,7633,7669,7699
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13451,13615,13818,13922,14134,14267,14413,14493,14634,14742,14821,14920,15008,15087,15103,15139,15251,15319,15361,15493,15536,15591,15648,15729,15804,15925,16061,16161,16188,16277,16362,16407,16484,16562,16626,16644,16686,16802,16931,17029


In [15]:
df1.isnull().sum()

state       188
country       0
lat           0
long          0
1/22/20       0
           ... 
12/20/20      0
12/21/20      0
12/22/20      0
12/23/20      0
12/24/20      0
Length: 342, dtype: int64

In [16]:

world_map = folium.Map(location=[11,0],
   zoom_start=2, width = '100%', height='100%', position = 'relative',
   tiles='OpenStreetMap',
   max_zoom=4, min_zoom =2)

for i in range(0,len(df1)):
    folium.Circle(
        location=[df1.iloc[i]['lat'], df1.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(df1.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+df1.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(df1.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(df2.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(df2.iloc[i,-1]/(df1.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map